In [ ]:
import warnings
warnings.filterwarnings("ignore")
import copy
import pandas as pd
from tb_incubator.constants import set_project_base_path, QUANTILES, image_path
from tb_incubator.calibrate import (
    plot_output_ranges, get_quantile_outputs, calculate_loo_comparison, load_idata, extract_and_save_idata, run_model_for_scenario
)
from tb_incubator.plotting import get_combined_plot, overlay_plots
from tb_incubator.input import load_targets, load_param_info

project_paths = set_project_base_path("../tb_incubator/")
calib_out = project_paths["OUT_PATH"]

In [ ]:
targets = load_targets()
out_req = ["notification_log", "adults_prevalence_pulmonary_log"]
params= load_param_info()["value"]

In [ ]:
file_suffix = "25000d10000t_01"
file_names = [f"xpert_only_{file_suffix}",
              f"no_xpert_no_cdr_{file_suffix}",
              f"xpert_cdr_inside_{file_suffix}",
              f"cdr_inside_{file_suffix}"]

assumption_titles = {}
for i, name in enumerate(file_names):
    assumption_titles[name] = f"Assumption {i+1}"

In [ ]:
titles = [assumption_titles[name] for name in file_names]
quantile_outputs = get_quantile_outputs(file_names, calib_out)
quantile_outs = quantile_outputs['xpert_only_25000d10000t_01']

In [ ]:
plot_indicators = {
    'notification': ['notification'],
    'adults_prev': ['adults_prevalence_pulmonary'], 
    'incidence': ['incidence'],
    'ltbi': ['percentage_latent'],
    'mortality': ['mortality'],
    'pdetect': ['passive_notification_raw']
}

plots = {}
for indicator_key, indicator_list in plot_indicators.items():
    plots[indicator_key] = {}
    for name in file_names:
        plots[indicator_key][name] = plot_output_ranges(
            quantile_outputs[name], targets, indicator_list, 1, 2015, 2035, 2013, 
            show_legend=True, show_target_data=False, show_title=False
        )


In [ ]:
indicators = ['notification', 'adults_prev', 'incidence', 'mortality']
assump_keys = ['xpert_only_25000d10000t_01', 'no_xpert_no_cdr_25000d10000t_01']

overlay_results = {}
for indicator in indicators:
    plot_list = [plots[indicator][key] for key in assump_keys]
    overlay_results[indicator] = overlay_plots(plot_list)

inc_plot_overlay = overlay_results['incidence']
mort_plot_overlay = overlay_results['mortality']
xpert_overlay = [inc_plot_overlay, mort_plot_overlay]

xpert_comp = get_combined_plot(
    plot_list=xpert_overlay, n_cols=2, subplot_titles=None, shared_yaxes=False, shared_xaxes=False
)
xpert_comp

#xpert_countf.write_image(image_path / f'xpert_countfact.png', scale=3)


In [ ]:
plot_list = [plots['pdetect']['xpert_only_25000d10000t_01'], plots['notification']['xpert_only_25000d10000t_01']]
overlay_plots(plot_list, update_layout=True, yaxis_title='No. of TB patient commenced treatment')

In [ ]:
plots_target = {}
for indicator_key, indicator_list in plot_indicators.items():
    plots_target[indicator_key] = {}
    for name in file_names:
        plots_target[indicator_key][name] = get_combined_plot(
                plot_list=list(plots[f'{indicator_key}'].values()),
                n_cols=2,
                subplot_titles=titles,
                shared_yaxes=True,
                shared_xaxes=False
        )

# save combined plots
# notif_combined = plots_target['notification']['xpert_only_25000d10000t_01']
# notif_combined.write_image(image_path / f'notifications_comp_{file_suffix}.png', scale=3)


In [ ]:
model_configs = {
    f'xpert_only_{file_suffix}': { # cdr is used outside detection function (only when calculating notification)
        'xpert_improvement': True,
        'apply_cdr_within_model': False,
    },
    f'no_xpert_no_cdr_{file_suffix}': {
        'xpert_improvement': False,
        'apply_cdr_within_model': False,
    },
    f'xpert_cdr_inside_{file_suffix}': {
        'xpert_improvement': True,
        'apply_cdr_within_model': True,
    },
    f'cdr_inside_{file_suffix}': {
        'xpert_improvement': False,
        'apply_cdr_within_model': True,
    },
}


In [ ]:
inference_data_dict = load_idata(calib_out, model_configs)
extract_and_save_idata(inference_data_dict, calib_out)
xpert_outputs = run_model_for_scenario(params, calib_out, model_configs, QUANTILES)

In [ ]:
loo_results = calculate_loo_comparison(xpert_outputs)
loo_results.to_csv('results_table_loo.csv', index=True)